# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd

In [ ]:
current_dir = %pwd
timestamp = os.path.basename(current_dir)

In [ ]:
import sys
sys.path.append('../../')
%cd '../../'

In [ ]:
from src.utils.trade_analysis import TradeAnalysis

%reload_ext autoreload
%autoreload 2

In [ ]:
ta = TradeAnalysis(timestamp)

In [ ]:
ex1_trades = ta.get_coincheck_trades_df()
ex2_trades = ta.get_liquid_trades_df()

In [ ]:
len(ex1_trades), len(ex2_trades)

In [ ]:
ex1_ticks = ta.get_coincheck_ticks_df()
ex2_ticks = ta.get_liquid_ticks_df()

In [ ]:
ex1_ticks.head(5)

In [ ]:
trades = []

for ex1_row, ex2_row in zip(ex1_trades.iterrows(), ex2_trades.iterrows()):
    ex1_item = ex1_row[1]
    ex2_item = ex2_row[1]
    
    ex1_side = ex1_item["side"]
    ex1_timestamp = ex1_item["datetime"]
    ex1_amount = ex1_item["amount"]
    ex1_actual_price = ex1_item["price"]
    ex1_actual_rate = ex1_item["rate"]

    ex2_side = ex2_item["side"]
    ex2_timestamp = ex2_item["datetime"]
    ex2_amount = ex2_item["amount"]
    ex2_actual_price = ex2_item["price"]
    ex2_actual_rate = ex2_item["rate"]
    
    try:
        ex1_tick = ex1_ticks.loc[ex1_timestamp]
    except Exception:
        timestamp = ex1_timestamp + datetime.timedelta(seconds=1)
        try:
            ex1_tick = ex1_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex1_tick = ex1_ticks.loc[timestamp2]

    try:
        ex2_tick = ex2_ticks.loc[ex2_timestamp]
    except Exception:
        timestamp = ex2_timestamp + datetime.timedelta(seconds=1)
        try:
            ex2_tick = ex2_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex2_tick = ex2_ticks.loc[timestamp2]

    if ex1_side == "sell":
        ex1_tick_rate = ex1_tick.bid
    else:
        ex1_tick_rate = ex1_tick.ask
    if ex2_side == "sell":
        ex2_tick_rate = ex2_tick.bid
    else:
        ex2_tick_rate = ex2_tick.ask
        
    ex1_expected_price = ex1_tick_rate * ex1_amount
    ex2_expected_price = ex2_tick_rate * ex2_amount

    
    if ex1_side == "buy":
        ex1_expected_price = ex1_expected_price * -1
        ex1_actual_price = ex1_actual_price * -1
    if ex2_side == "buy":
        ex2_expected_price = ex2_expected_price * -1
        ex2_actual_price = ex2_actual_price * -1
        
    expected_profit = sum([ex1_expected_price, ex2_expected_price])
    actual_profit = sum([ex1_actual_price, ex2_actual_price])

    trade = {
        "timestamp": ex1_timestamp, 
        "ex1_side": ex1_side,
        "ex1_expected_rate": ex1_tick_rate,
        "ex1_actual_rate": ex1_actual_rate,
        "ex1_expected_price": ex1_expected_price,
        "ex1_actual_price": ex1_actual_price,
        "ex2_side": ex2_side,
        "ex2_expected_rate": ex2_tick_rate,
        "ex2_actual_rate": ex2_actual_rate,
        "ex2_expected_price": ex2_expected_price,
        "ex2_actual_price": ex2_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [ ]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [ ]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [ ]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

In [ ]:
pd.set_option('display.max_rows', None)
profit_df

## coincheck buyにおける売却コインの差分分析

In [ ]:
ex1_buy_trades = ex1_trades[ex1_trades["side"] == "buy"]

In [ ]:
ex1_buy_amount = ex1_buy_trades["amount"]
min(ex1_buy_amount), max(ex1_buy_amount), np.mean(ex1_buy_amount)

In [ ]:
from decimal import Decimal
num = 0.006 - np.mean(ex1_buy_amount)
"{0:.10f}".format(Decimal(num))

この価格差によって生じた資金移動は

In [ ]:
num * len(ex1_buy_trades) * np.mean(ex1_ticks["ask"])

## rateの誤差比較

In [ ]:
ex1_rate_df = trade_df[["ex1_side", "ex1_expected_rate", "ex1_actual_rate"]].copy()
ex1_rate_df["ex1_rate_diff"] = ex1_rate_df["ex1_expected_rate"] - ex1_rate_df["ex1_actual_rate"]

In [ ]:
ex1_rate_df

In [ ]:
ex1_diff = ex1_rate_df["ex1_rate_diff"]
max(ex1_diff), min(ex1_diff), np.mean(abs(ex1_diff)), np.std(ex1_diff)

In [ ]:
ex2_rate_df = trade_df[["ex2_side", "ex2_expected_rate", "ex2_actual_rate"]].copy()
ex2_rate_df["ex2_rate_diff"] = ex2_rate_df["ex2_expected_rate"] - ex2_rate_df["ex2_actual_rate"]

In [ ]:
ex2_diff = ex2_rate_df["ex2_rate_diff"]
max(ex2_diff), min(ex2_diff), np.mean(abs(ex2_diff)), np.std(ex2_diff)